Exercise 2

* Download the video from here: https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4 (also available under Exercise Materials on the main course page)
* Compute optical flow with Lucas Kanade and GoodFeaturesToTrack (hint, hint)
* Instead of colored tracks, make your program output those cool optical flow arrows (hint: you need dense optical flow for this, so use Färnebäck for this):

In [ ]:
# Importing libraries & packages
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import time
import glob
import sys

# Define data path + output folder
data = "../Data/slow_traffic_small.mp4"
output = "output/"

Optical flow with goodFeatureToTrack

In [ ]:
cap = cv.VideoCapture(data)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]

    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)

    cv.imshow('frame', img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv.destroyAllWindows()

Dense Optical Flow with Farneback

In [5]:
cap = cv.VideoCapture(data)
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    for y in range(0, frame2.shape[0], 10):
        for x in range(0, frame2.shape[1], 10):
            flow_at_point = np.int32(np.round(flow[y, x] * 2.5))
            line_start = (x, y)
            line_end = (x+flow_at_point[0], y+flow_at_point[1])
            cv.line(frame2, line_start, line_end, (0, 0, 255), 1)
    
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    
    cv.imshow('frame2', frame2)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite(output+'opticalfb.png', frame2)
        cv.imwrite(output+'opticalhsv.png', bgr)
        prvs = next
        
cv.destroyAllWindows()

Exercise Solution:

In [ ]:
cap = cv.VideoCapture(data)

_, current_frame = cap.read()
current_frame_gray = cv.cvtColor(current_frame, cv.COLOR_BGR2GRAY)

while True:
    ret, next_frame = cap.read()
    next_frame_gray = cv.cvtColor(next_frame, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(current_frame_gray, next_frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    for y in range(0, current_frame.shape[0], 10):
        for x in range(0, current_frame.shape[1], 10):
            flow_at_point = np.int32(np.round(flow[y, x] * 5))
            line_start = (x, y)
            line_end = (x+flow_at_point[0], y+flow_at_point[1])
            cv.line(current_frame, line_start, line_end, (0, 0, 255), 1)

    cv.imshow('Optical flow', current_frame)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    current_frame = next_frame
    current_frame_gray = next_frame_gray
    
cv.destroyAllWindows()